In [28]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import shapely as shp

from itertools import compress

# make sure we are at the top of the repo
wd = subprocess.check_output('git rev-parse --show-toplevel', shell = True)
os.chdir(wd[:-1]) #-1 removes \n

In [2]:
# read in nyc census block shapefiles
nycshp = gp.read_file('data/nycb2010_16c/nycb2010.shp')

# set up cooridnate system
nycshp = nycshp.to_crs(epsg=2263) # epsg=4326: lat/on | 26918: NAD83/UTM zone 18N | epsg=2263 is US feet

# we only need manhattan 
manshp = nycshp[nycshp.BoroName == 'Manhattan']

# add state code 36 and leading digits for county code 06
manshp['BCTCB2010'] = '3606' + manshp['BCTCB2010']

# keep only what we need
manshp = manshp[['BCTCB2010','geometry']]

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
# read in 2014 citibike data
citi = pd.DataFrame()
for m in list(compress(os.listdir('data'), [('citibike' in i) for i in os.listdir('data')])):
    citinew = pd.read_csv('data/'+m)
    citi = citi.append(citinew, ignore_index = True)

In [43]:
# dataframe for stations    
citi_docks = citi[['start station id','start station name','start station latitude','start station longitude']]
citi_docks.columns = ['station_id','name','lat','lon']
citi_docks.drop_duplicates(inplace = True)

In [51]:
# dataframe for rides per station
citi_rides = citi[['start station id', 'tripduration']] # keep duration because we need a column to count with
citi_rides.columns = ['station_id','n_rides']
citi_rides = citi_rides.groupby('station_id').count()

In [57]:
# merge number of rides into station detail table
citi_docks = citi_docks.merge(citi_rides, left_on = 'station_id', right_index = True)

In [59]:
citi_docks.head()

,station_id,name,lat,lon,n_rides
0,2009,Catherine St & Monroe St,40.711174,-73.996826,13610
1,536,1 Ave & E 30 St,40.741444,-73.975361,33802
2,228,E 48 St & 3 Ave,40.754601,-73.971879,21674
3,519,Pershing Square N,40.751884,-73.977702,92137
525203,519,E 42 St & Vanderbilt Ave,40.752416,-73.978370,92137
